<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Structure-du-réseau" data-toc-modified-id="Structure-du-réseau-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Structure du réseau</a></span></li><li><span><a href="#Configuration-du-processus-d'apprentissage" data-toc-modified-id="Configuration-du-processus-d'apprentissage-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Configuration du processus d'apprentissage</a></span><ul class="toc-item"><li><span><a href="#Principales-fonctions-de-coût" data-toc-modified-id="Principales-fonctions-de-coût-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Principales fonctions de coût</a></span><ul class="toc-item"><li><span><a href="#Pour-les-problèmes-de-classification" data-toc-modified-id="Pour-les-problèmes-de-classification-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Pour les problèmes de classification</a></span></li><li><span><a href="#Pour-les-problèmes-de-régression" data-toc-modified-id="Pour-les-problèmes-de-régression-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Pour les problèmes de régression</a></span></li></ul></li><li><span><a href="#Algorithmes-d'optimisation" data-toc-modified-id="Algorithmes-d'optimisation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Algorithmes d'optimisation</a></span></li><li><span><a href="#Métrique-d'évaluation" data-toc-modified-id="Métrique-d'évaluation-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Métrique d'évaluation</a></span><ul class="toc-item"><li><span><a href="#Problèmes-de-régression" data-toc-modified-id="Problèmes-de-régression-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Problèmes de régression</a></span></li><li><span><a href="#Problèmes-de-classification" data-toc-modified-id="Problèmes-de-classification-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Problèmes de classification</a></span></li></ul></li></ul></li><li><span><a href="#Entraînement-et-évaluation-du-réseau" data-toc-modified-id="Entraînement-et-évaluation-du-réseau-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Entraînement et évaluation du réseau</a></span></li></ul></div>

# Mise en oeuvre d'un réseau neuronal avec Keras

<br>

Keras est une interface de programmation (API: Application Programming Interface) simplifiant l'utilisation de la plateforme de machine learning *Tensor Flow*. C'est l'outil que nous allons utiliser pour construire et entraîner des réseaux neuronaux. La documentation de Keras est disponible à l'adresse suivante: https://keras.io/api/.

<br>

Vous trouverez ci-dessous les éléments de base qui vous permettront d'utiliser keras pour construire et entraîner un réseau de neurones simple. Ces éléments sont directement tirés de la documentation de Keras.

<br>



## Structure du réseau

<br>

Le modèle le plus simple que l'on peut construire est le modèle <code>Sequential</code> constitué d'un empilement de couches connectées successivement les unes aux autres. Pour créer un modèle (initialement vide), on doit créer une instance de la classe <code>Sequential</code> 

<br>

In [2]:
from tensorflow.keras.models import Sequential

model = Sequential()

<br>

Pour ajouter une couche de neurones au modèle, il faut appeler la méthode <code>add()</code> en indiquant en argument le type de couche que l'on veut mettre dans le réseau.

Pour une couche "standard" totalement connectée on utilise la classe <code>Dense(units=nb_units, activation=activation_function)</code> où *nb_units* correspond au nombre de neurones dans la couche et *activation_function* correspond à la fonction d'activation des neurones de cette couche ('relu', 'sigmoid', 'tanh', 'softmax', ...).

Supposons que l'on souhaite construire un réseau constitué de 2 couches ayant la structure suivante:
- couche 1: 64 unités, activation de type 'relu'
- couche 2: 4 unités, activation de type 'softmax'

et que les données qui vont être fournies en entrée du réseau soient de dimension $d=4$. Pour créer la structure de ce réseau on va alors appeler les commandes suivantes:

<br>


In [3]:
from tensorflow.keras.layers import Dense

model.add(Dense(units=64, activation='relu', input_shape=(4,)))
model.add(Dense(units=3, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 515 (2.01 KB)
Trainable params: 515 (2.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


<br>

<img src="./schema-NN.png" width=400 height=300 />

<br>

L'indication du paramètre <code>input_shape=(4,)</code> n'est pas indispensable mais elle permet d'utiliser la fonction <code>summary()</code> qui affiche la structure du réseau ainsi que le nombre de paramètres dans chaque couche.

<br>

Les objets de type *layer* constituent la base d'un réseau neuronal. Il en existe de nombreux types. Pour plus d'informations, voir https://keras.io/api/layers/ .

<br>

<br>

## Configuration du processus d'apprentissage

<br>

Une fois que la structure du réseau est définie, on peut configurer le processus d'apprentissage à l'aide de la méthode <code>compile()</code>. Lors de cette étape, il faut en particulier définir:
- la fonction de coût à minimiser
- l'algorithme d'optimisation
- la métrique utilisée pour évaluer la performance du modèle

<br>

### Principales fonctions de coût

<br>

Voir https://keras.io/api/losses/.

<br>

#### Pour les problèmes de classification

<br>

- **binary_crossentropy**: entropie croisée pour les problème de classification binaire (y = 0 ou 1)
- **categorical_crossentropy**: entropie croisée pour les problèmes multi-classes; les étiquettes doivent être fournies dans une représentation de type *one-hot* (voir ci-dessous)
- **sparse_categorical_crossentropy**: entropie croisée pour les problèmes multi-classes; les étiquettes doivent être fournies sous la forme d'un entier (compris entre 1 et K s'il y a K classes)

#### Représentation *one-hot*

Pour représenter 1 classe on peut créer une variable dont la valeur sera un nombre entier compris entre 1 et K où K est le nombre de classes. L'inconvénient de cettre forme de représentation est qu'elle introduit une notion d'ordre entre les différentes classes. Pour éviter cela on peut utiliser une autre forme de représentation appelée *one-hot*. Pour cela, on va créer une variable vectorielle possédant K composantes pouvant prendre la valeur 0 ou 1. Pour une donnée appartenant à la classe $m$, toutes les composantes seront nulles sauf la composante $m$ qui aura la valeur 1. Par exemple, dans le cas d'un problème où le nombre de classes est 5, pour une donnée de la classe 4 la variable représentant la classe prendra la valeur (0,0,0,1,0).


#### Pour les problèmes de régression

<br>

- **mean_squared_error**
- **mean_absolute_error**

<br>

### Algorithmes d'optimisation

Plusieurs algorithmes d'optimisation peuvent être utilisés (https://keras.io/api/optimizers/). On trouve notamment:
- **SGD**: descente de gradient stochastique avec momentum
- **RMSprop**: descente de gradient stochastique à pas adaptatif
- **Adam**: descente de gradient stochastique avec momentum et pas adaptatif

<br>

### Métrique d'évaluation

Un grand nombre de métriques peuvent être utilisées (https://keras.io/api/metrics/).

#### Problèmes de régression

Pour les problèmes de régression, on utilise très souvent l'erreur quadratique moyenne (**mean_squared_error**) ou l'erreur absolue moyenne (**mean_absolute_error**).

#### Problèmes de classification

Pour les problèmes de classification, la métrique la plus simple est **accuracy** qui renvoie le pourcentage de prédictions correctes.

<br>

La configuration la plus simple peut être effectuée à l'aide d'une commande du type:

<br>

In [4]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

<br>

La commande précédente utilise les paramètres par défaut de l'optimiseur. Il est cependant évidemment possible de configurer plus finement le processus en choisissant les paramètres de l'optimiseur:

<br>

In [5]:
from tensorflow import keras

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

<br>

## Entraînement et évaluation du réseau

<br>

voir: https://keras.io/api/models/model_training_apis/

<br>
Pour entraîner et évaluer le modèle ainsi créé il faut disposer d'un jeu de données d'entraînement et d'un jeu de test. Certains datasets sont directement intégrés à Keras mais il est évidemment possible d'utiliser d'autres jeux de données. Dans cet exemple nous allons utiliser les données du dataset Iris disponible dans scikit-learn.

<br>

In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

dataset = load_iris()
X = dataset['data']
y = dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

<br>

Comme avec la librairie scikit-learn, l'optimisation du modèle avec les données d'entraînement s'effectue grâce à la méthode <code>fit()</code>. En plus des données d'entraînement, les principaux paramètres qui peuvent passés en argument sont
- epochs : nombre de cycles où l'ensemble des données d'entraînement sera utilisé pour minimiser la fonction de coût
- batch_size : taille des mini-lots utilisés pour la descente de gradient
- validation_split : fraction des données d'entraînement utilisée pour réaliser une estimation de l'erreur de généralisation

Au cours de l'entraînement, le programme affiche la valeur de la fonction de coût et de la métrique d'évaluation calculées sur les données d'entraînement (*loss* et *accuracy*) ainsi que la valeur de ces grandeurs mesurée sur les données de validation (*val_loss* et *val_accuracy*).

<br>


In [7]:
model.fit(X_train, y_train, epochs=20, batch_size=10, validation_split=0.2)

Epoch 1/20
9/9 [==============================] - 0s 18ms/step - loss: 0.8613 - accuracy: 0.6517 - val_loss: 0.7839 - val_accuracy: 0.6522
Epoch 2/20
9/9 [==============================] - 0s 4ms/step - loss: 0.5743 - accuracy: 0.7978 - val_loss: 0.6431 - val_accuracy: 0.6957
Epoch 3/20
9/9 [==============================] - 0s 4ms/step - loss: 0.4260 - accuracy: 0.8427 - val_loss: 0.5622 - val_accuracy: 0.7826
Epoch 4/20
9/9 [==============================] - 0s 3ms/step - loss: 0.3605 - accuracy: 0.8764 - val_loss: 0.4884 - val_accuracy: 0.8261
Epoch 5/20
9/9 [==============================] - 0s 5ms/step - loss: 0.3187 - accuracy: 0.8764 - val_loss: 0.4302 - val_accuracy: 0.8261
Epoch 6/20
9/9 [==============================] - 0s 3ms/step - loss: 0.2893 - accuracy: 0.8764 - val_loss: 0.3835 - val_accuracy: 0.8261
Epoch 7/20
9/9 [==============================] - 0s 4ms/step - loss: 0.2680 - accuracy: 0.8876 - val_loss: 0.3569 - val_accuracy: 0.8261
Epoch 8/20
9/9 [=================

<br>

La méthode <code>evaluate()</code> renvoie la valeur de la fonction de coût et de la métrique d'évaluation calculées sur les données fournies en argument.

<br>


In [8]:
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=len(y_test))

print("\nloss:", loss_and_metrics[0])
print("accuracy:", loss_and_metrics[1])

1/1 [==============================] - 0s 22ms/step - loss: 0.1972 - accuracy: 0.9737

loss: 0.19717200100421906
accuracy: 0.9736841917037964


<br>

La méthode <code>predict()</code> permet de prédire l'étiquette associée à de nouvelles données. Dans le cas d'un problème de classification cette prédiction correspond généralement à la probabilité d'appartenir à chacune des classes.

<br>


In [9]:
import numpy as np

predictions = model.predict(X_test, batch_size=len(y_test))

for i in range(len(y_test)):
    if y_test[i]==np.argmax(predictions[i]):
        print(y_test[i], '   ', predictions[i])
    else:
        print(y_test[i], '   ', predictions[i], '     ERROR')

1/1 [==============================] - 0s 48ms/step
2     [2.5036783e-04 1.6231753e-02 9.8351789e-01]
1     [0.00425645 0.9527472  0.04299627]
0     [9.98232722e-01 1.65977026e-03 1.07501575e-04]
2     [1.5280371e-04 4.7996797e-02 9.5185041e-01]
0     [9.9511188e-01 4.6570064e-03 2.3106391e-04]
2     [1.7596326e-04 8.3384821e-03 9.9148554e-01]
0     [9.9650562e-01 3.3008417e-03 1.9354737e-04]
1     [0.01527964 0.59790653 0.38681382]
1     [0.00548419 0.6206866  0.37382925]
1     [0.02469966 0.8613345  0.11396587]
2     [0.00231178 0.3587607  0.63892746]
1     [0.03378204 0.6577654  0.30845258]
1     [0.01665686 0.8138481  0.16949508]
1     [0.0071499  0.56324416 0.42960593]
1     [0.01681823 0.6468873  0.33629447]
0     [9.983188e-01 1.585947e-03 9.532721e-05]
1     [0.01848143 0.5972433  0.3842753 ]
1     [0.01974134 0.8599509  0.12030769]
0     [9.8172194e-01 1.7820276e-02 4.5770465e-04]
0     [9.9650443e-01 3.2739670e-03 2.2171738e-04]
2     [0.00236267 0.0920568  0.9055805 ]
1     

<br>

## Utilisation des *callbacks*

<br>

Au cours de la phase d'entraînement du réseau il est possible d'activer certaines fonctions, en particulier pour effectuer une sauvegarde régulière du modèle ou pour implémenter une stratégie d'arrêt anticipé (*early stopping*). Sous Keras ces fonctions sont appelées **callbacks** (https://keras.io/api/callbacks/).

<br>

La cellule ci-dessous crée 2 objets de type *callback*. Le premier (*cb_sauvegarde*) va permettre de sauvegarder le modèle dans le fichier *fichier_de_sauvegarde.h5* à la fin de chaque époque d'entraînement. Lorsque l'entraînement du réseau prend beaucoup de temps, ceci permet de se prémunir contre des problèmes tels qu'une coupure d'électricité ou un plantage de l'ordinateur. L'option *save_best_only* indique à l'algorithme d'effectuer la sauvegarde uniquement lorsque la métrique d'évaluation est meilleure que lors de la sauvegarde précédente. Cette option permet d'entraîner l'algorithme avec un nombre d'époques très grand sans craindre les problèmes de sur-apprentissage puisque le modèle cesse d'être sauvegardé lorsque la métrique d'évaluation commence à se détériorer.

Le second callback (*cb_early_stopping*) permet d'interrompre l'entraînement si la métrique d'évaluation cesse de s'améliorer pendant 3 époques consécutives.

<br>

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

cb_sauvegarde = ModelCheckpoint(filepath='fichier_de_sauvegarde.h5', save_best_only=True)
cb_earlystopping = EarlyStopping(monitor="val_accuracy", patience=3)

In [11]:
new_model = Sequential()
new_model.add(Dense(units=64, activation='relu', input_shape=(4,)))
new_model.add(Dense(units=3, activation='softmax'))

new_model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True),
                  metrics=['accuracy'])

new_model.fit(X_train, y_train,
              epochs=20, 
              batch_size=10, 
              validation_split=0.2, 
              callbacks=[cb_sauvegarde, cb_earlystopping])

Epoch 1/20
9/9 [==============================] - 0s 34ms/step - loss: 0.9969 - accuracy: 0.4270 - val_loss: 0.8480 - val_accuracy: 0.5217
Epoch 2/20
1/9 [==>...........................] - ETA: 0s - loss: 0.6898 - accuracy: 0.8000

C:\Users\lucas\anaconda3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


9/9 [==============================] - 0s 6ms/step - loss: 0.6040 - accuracy: 0.7865 - val_loss: 0.6788 - val_accuracy: 0.7391
Epoch 3/20
9/9 [==============================] - 0s 4ms/step - loss: 0.4459 - accuracy: 0.8202 - val_loss: 0.6098 - val_accuracy: 0.7391
Epoch 4/20
9/9 [==============================] - 0s 4ms/step - loss: 0.3756 - accuracy: 0.8539 - val_loss: 0.5376 - val_accuracy: 0.8261
Epoch 5/20
9/9 [==============================] - 0s 4ms/step - loss: 0.3340 - accuracy: 0.8764 - val_loss: 0.4692 - val_accuracy: 0.8261
Epoch 6/20
9/9 [==============================] - 0s 4ms/step - loss: 0.3053 - accuracy: 0.8764 - val_loss: 0.4203 - val_accuracy: 0.8261
Epoch 7/20
9/9 [==============================] - 0s 5ms/step - loss: 0.2807 - accuracy: 0.8764 - val_loss: 0.3934 - val_accuracy: 0.8261


<br>

On peut également sauvegarder un modèle sans utiliser les callbacks. Une fois l'entraînement terminé, il suffit pour cela d'appeler la fonction <code>.save()</code>. Une fois qu'un modèle a été sauvegardé, il est naturellement possible de le charger à l'aide de la fonction <code>.load_model()</code> afin de l'utiliser ou de continuer l'entraînement.

<br>

#### En conclusion

<br>

Nous avons présenté dans ce notebook les éléments de base de l'API Keras afin que vous puissiez commencer à l'utiliser. Cette plateforme offre de nombreuses autres fonctionnalités et permet de construire des modèles extrêmement complexes. Les liens vers la documentation de Keras doivent vous permettre de mieux comprendre le fonctionnement de cette API et d'en exploiter les possibilités.

<br>